VGG ( Visual Geometry Group ) is a widely used type of neural network architecture a CNN, known for its unifrom design.
The numbers in VGG16 or VGG19 refer to the depth of neural network.

In [ ]:
# there are 6 block of network
# block 1 : 2 Conv layer + ReLU and a Pooling
# block 2 : 2 Conv Layer + ReLU and a pooling
# block 3 : 3 Conv Layer + ReLU and a Pooling
# block 4 : 3 Conv Layer + ReLU and a Pooling
# block 5 : 3 Conv Layer + ReLU and a Pooling
# block 6 : 3 FC layer

In [18]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data.sampler import SubsetRandomSampler # to ranomly sample data
from torchsummary import summary

In [2]:
# Device Config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
root_input_dataset_dir =  "/content/drive/My Drive/input_dataset"

In [9]:
# Data Loader
def data_loader( data_dir, batch_size , random_seed = 42, shuffle = True, valid_size=0.1 , test = False):

  # Normalization
  # define transforms
  # make the size of images uniform
  transform = transforms.Compose([
      transforms.Resize((227,227)),
      transforms.ToTensor(),
      transforms.Normalize(
      mean = [ 0.4914, 0.4822, 0.4465],
      std = [ 0.2023, 0.1994, 0.2010])
    ])

  if test:
    dataset = datasets.CIFAR100( root = data_dir, train = False, download = True, transform = transform)
    data_loader = torch.utils.data.DataLoader( dataset = dataset, batch_size = batch_size, shuffle = shuffle)
    return data_loader

  train_dataset = datasets.CIFAR100( root = data_dir, download = True , train = True, transform=transform)
  val_dataset = datasets.CIFAR100( root = data_dir, download = True , train = True, transform=transform)

  num_train = len(train_dataset)
  indices = list( range(num_train))
  split = int(np.floor(valid_size*num_train))

  if shuffle:
    np.random.seed(random_seed)
    np.random.shuffle(indices) # for shuffleing the dataset

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    val_data_loader = torch.utils.data.DataLoader( dataset = val_dataset, batch_size = batch_size, sampler = train_sampler )
    train_data_loader = torch.utils.data.DataLoader( dataset = train_dataset, batch_size = batch_size, sampler = train_sampler )

    return (train_data_loader, val_data_loader)


In [10]:
train_loader ,valid_loader = data_loader( root_input_dataset_dir, batch_size = 64 )

Files already downloaded and verified
Files already downloaded and verified


In [11]:
len(train_loader)

704

In [12]:
test_loader = data_loader( root_input_dataset_dir, batch_size = 64, test = True)

Files already downloaded and verified


In [13]:
class VGG16(nn.Module):

  def __init__(self, num_classes=10):
    super(VGG16, self).__init__()

    self.layer1 = nn.Sequential(
        nn.Conv2d( 3, 64, kernel_size = 3 , stride = 1, padding =1 ),
        nn.BatchNorm2d(64),
        nn.ReLU()
    )

    self.layer2 = nn.Sequential(
        nn.Conv2d(64, 64, kernel_size = 3 ,stride = 1, padding =1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size =2, stride =2)
    )

    self.layer3 = nn.Sequential(
        nn.Conv2d(64, 128, kernel_size = 3 ,stride = 1, padding =1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
    )

    self.layer4 = nn.Sequential(
        nn.Conv2d(128, 128, kernel_size = 3 ,stride = 1, padding =1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size =2, stride =2)
    )

    self.layer5 = nn.Sequential(
        nn.Conv2d(128, 256, kernel_size = 3 ,stride = 1, padding =1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
    )

    self.layer6 = nn.Sequential(
        nn.Conv2d(256, 256, kernel_size = 3 ,stride = 1, padding =1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
    )

    self.layer7 = nn.Sequential(
        nn.Conv2d(256, 256, kernel_size = 3 ,stride = 1, padding =1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size =2, stride =2)
    )

    self.layer8 = nn.Sequential(
        nn.Conv2d(256, 512, kernel_size = 3 ,stride = 1, padding =1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
    )

    self.layer9 = nn.Sequential(
        nn.Conv2d(512, 512, kernel_size = 3 ,stride = 1, padding =1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
    )

    self.layer10 = nn.Sequential(
        nn.Conv2d(512, 512, kernel_size = 3 ,stride = 1, padding =1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size =2, stride =2)
    )

    self.layer11 = nn.Sequential(
        nn.Conv2d(512, 512, kernel_size = 3 ,stride = 1, padding =1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
    )

    self.layer12 = nn.Sequential(
        nn.Conv2d(512, 512, kernel_size = 3 ,stride = 1, padding =1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
    )

    self.layer13 = nn.Sequential(
        nn.Conv2d(512, 512, kernel_size = 3 ,stride = 1, padding =1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size =2, stride =2)
    )

    self.fc1 = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(7*7*512, 4096),
        nn.ReLU()
    )

    self.fc2 = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(4096, 4096),
        nn.ReLU()
    )

    self.fc3 = nn.Sequential(
        nn.Linear(4096, num_classes)
    )


  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = self.layer5(out)
    out = self.layer6(out)
    out = self.layer7(out)
    out = self.layer8(out)
    out = self.layer9(out)
    out = self.layer10(out)
    out = self.layer11(out)
    out = self.layer12(out)
    out = self.layer13(out)
    out = out.view(out.size(0), -1)
    out = self.fc1(out)
    out = self.fc2(out)
    out = self.fc3(out)
    return out

In [14]:
# defining hyperparameter
num_classes= 100
num_epochs = 20
batch_size = 16
learning_rate =0.001


In [15]:
model = VGG16(num_classes).to(device)

In [19]:
summary(model, (3, 227,227))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 227, 227]           1,792
       BatchNorm2d-2         [-1, 64, 227, 227]             128
              ReLU-3         [-1, 64, 227, 227]               0
            Conv2d-4         [-1, 64, 227, 227]          36,928
       BatchNorm2d-5         [-1, 64, 227, 227]             128
              ReLU-6         [-1, 64, 227, 227]               0
         MaxPool2d-7         [-1, 64, 113, 113]               0
            Conv2d-8        [-1, 128, 113, 113]          73,856
       BatchNorm2d-9        [-1, 128, 113, 113]             256
             ReLU-10        [-1, 128, 113, 113]               0
           Conv2d-11        [-1, 128, 113, 113]         147,584
      BatchNorm2d-12        [-1, 128, 113, 113]             256
             ReLU-13        [-1, 128, 113, 113]               0
        MaxPool2d-14          [-1, 128,

In [20]:
# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD( model.parameters(), lr = learning_rate)

In [21]:
total_step = len(train_loader)

In [ ]:
for epoch in range(num_epochs):
  model.train()
  for i, (images, labels) in enumerate(train_loader):
    images, labels = images.to(device), labels.to(device)


    predictions = model(images)
    loss = criterion(predictions)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))
  model.eval()
  with torch.no_grad():

    correct = 0
    total =0

    for images, labels in valid_loader:
      images, labels = images.to(device), labels.to(device)
      predictions = model(images)
      _, predicted = torch.max(predictions.data.item(), 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
      del images, labels, predictions

    print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total))


In [ ]:
# for test dataset
with torch.no_grad():

    correct = 0
    total =0

    for images, labels in test_loader:
      images, labels = images.to(device), labels.to(device)
      predictions = model(images)
      _, predicted = torch.max(predictions.data.item(), 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
      del images, labels, predictions

    print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total))
